In [1]:
import sqlite3
import urllib.request
import pandas as pd

url = "https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite"
urllib.request.urlretrieve(url, "Chinook.sqlite")

conn = sqlite3.connect("Chinook.sqlite")
cursor = conn.cursor()

In [2]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in Chinook database:")
pd.DataFrame(tables, columns=['Table Name'])

Tables in Chinook database:


,Table Name
0,Album
1,Artist
2,Customer
3,Employee
4,Genre
5,Invoice
6,InvoiceLine
7,MediaType
8,Playlist
9,PlaylistTrack


In [4]:
# Query 1: Simple artist query
query1 = "SELECT * FROM Artist WHERE Name LIKE 'A%' LIMIT 5;"
df1 = pd.read_sql_query(query1, conn)
print("\nArtists starting with 'A':")
display(df1)

# Query 2: Tracks with their album and artist
query2 = """
SELECT t.Name AS Track, a.Title AS Album, ar.Name AS Artist 
FROM Track t
JOIN Album a ON t.AlbumId = a.AlbumId
JOIN Artist ar ON a.ArtistId = ar.ArtistId
LIMIT 5;
"""
df2 = pd.read_sql_query(query2, conn)
print("\nTracks with album and artist info:")
display(df2)

# Query 3: Customers with their invoices
query3 = """
SELECT c.FirstName, c.LastName, COUNT(i.InvoiceId) AS InvoiceCount
FROM Customer c
LEFT JOIN Invoice i ON c.CustomerId = i.CustomerId
GROUP BY c.CustomerId
ORDER BY InvoiceCount DESC
LIMIT 5;
"""
df3 = pd.read_sql_query(query3, conn)
print("\nCustomers with most invoices:")
display(df3)


Artists starting with 'A':


,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains



Tracks with album and artist info:


,Track,Album,Artist
0,For Those About To Rock (We Salute You),For Those About To Rock We Salute You,AC/DC
1,Balls to the Wall,Balls to the Wall,Accept
2,Fast As a Shark,Restless and Wild,Accept
3,Restless and Wild,Restless and Wild,Accept
4,Princess of the Dawn,Restless and Wild,Accept



Customers with most invoices:


,FirstName,LastName,InvoiceCount
0,Luís,Gonçalves,7
1,Leonie,Köhler,7
2,François,Tremblay,7
3,Bjørn,Hansen,7
4,František,Wichterlová,7


In [5]:
# Join across Customer, Invoice, and InvoiceLine tables
three_table_query = """
SELECT 
    c.FirstName || ' ' || c.LastName AS CustomerName,
    i.InvoiceDate,
    t.Name AS TrackPurchased,
    t.UnitPrice
FROM 
    Customer c
JOIN 
    Invoice i ON c.CustomerId = i.CustomerId
JOIN 
    InvoiceLine il ON i.InvoiceId = il.InvoiceId
JOIN 
    Track t ON il.TrackId = t.TrackId
ORDER BY 
    i.InvoiceDate DESC
LIMIT 10;
"""

three_table_result = pd.read_sql_query(three_table_query, conn)
print("\nRecent purchases with customer, date, and track info:")
display(three_table_result)


Recent purchases with customer, date, and track info:


,CustomerName,InvoiceDate,TrackPurchased,UnitPrice
0,Manoj Pareek,2013-12-22 00:00:00,Hot Girl,1.99
1,Terhi Hämäläinen,2013-12-14 00:00:00,Higher Ground,0.99
2,Terhi Hämäläinen,2013-12-14 00:00:00,Secrets,0.99
3,Terhi Hämäläinen,2013-12-14 00:00:00,Eruption,0.99
4,Terhi Hämäläinen,2013-12-14 00:00:00,Dreams,0.99
5,Terhi Hämäläinen,2013-12-14 00:00:00,Eruption,0.99
6,Terhi Hämäläinen,2013-12-14 00:00:00,On Fire,0.99
7,Terhi Hämäläinen,2013-12-14 00:00:00,Year to the Day,0.99
8,Terhi Hämäläinen,2013-12-14 00:00:00,Fall To Pieces,0.99
9,Terhi Hämäläinen,2013-12-14 00:00:00,A Bencao E Outros,0.99


In [6]:

# First drop existing indexes that would help this query
cursor.executescript("""
DROP INDEX IF EXISTS IFK_InvoiceCustomerId;
DROP INDEX IF EXISTS IFK_InvoiceLineInvoiceId;
DROP INDEX IF EXISTS IFK_InvoiceLineTrackId;
""")
conn.commit()

# Query with expressions to prevent index usage
unindexed_query = """
SELECT 
    c.FirstName || ' ' || c.LastName AS CustomerName,
    i.InvoiceDate,
    t.Name AS TrackPurchased
FROM 
    Customer c
JOIN 
    Invoice i ON (c.CustomerId + 0) = (i.CustomerId + 0)
JOIN 
    InvoiceLine il ON (i.InvoiceId + 0) = (il.InvoiceId + 0)
JOIN 
    Track t ON (il.TrackId + 0) = (t.TrackId + 0)
ORDER BY 
    i.InvoiceDate DESC
LIMIT 10;
"""

unindexed_result = pd.read_sql_query(unindexed_query, conn)
print("\nRecent purchases (unindexed joins):")
display(unindexed_result)


Recent purchases (unindexed joins):


,CustomerName,InvoiceDate,TrackPurchased
0,Manoj Pareek,2013-12-22 00:00:00,Hot Girl
1,Terhi Hämäläinen,2013-12-14 00:00:00,Higher Ground
2,Terhi Hämäläinen,2013-12-14 00:00:00,Secrets
3,Terhi Hämäläinen,2013-12-14 00:00:00,Eruption
4,Terhi Hämäläinen,2013-12-14 00:00:00,Dreams
5,Terhi Hämäläinen,2013-12-14 00:00:00,Eruption
6,Terhi Hämäläinen,2013-12-14 00:00:00,On Fire
7,Terhi Hämäläinen,2013-12-14 00:00:00,Year to the Day
8,Terhi Hämäläinen,2013-12-14 00:00:00,Fall To Pieces
9,Terhi Hämäläinen,2013-12-14 00:00:00,A Bencao E Outros


In [8]:
# ## 5. Timing Unindexed Query (10 Runs)

# %%

import time 
def time_query(query, runs=10):
    times = []
    for i in range(runs):
        # Vary the query slightly to avoid caching
        varied_query = query.replace("LIMIT 10", f"LIMIT {10 + i}")
        start_time = time.time()
        pd.read_sql_query(varied_query, conn)
        end_time = time.time()
        times.append(end_time - start_time)
    return times

unindexed_times = time_query(unindexed_query)
avg_unindexed = sum(unindexed_times) / len(unindexed_times)

print(f"Unindexed query times (seconds): {[round(t, 4) for t in unindexed_times]}")
print(f"Average time: {avg_unindexed:.4f} seconds")

Unindexed query times (seconds): [0.4158, 0.2765, 0.2678, 0.2997, 0.2782, 0.2792, 0.2833, 0.3097, 0.2759, 0.2692]
Average time: 0.2955 seconds


In [9]:
# ## 6. Create Indexes for Performance

# %%
# Create indexes to optimize our query
cursor.executescript("""
CREATE INDEX IF NOT EXISTS idx_customer_id ON Customer(CustomerId);
CREATE INDEX IF NOT EXISTS idx_invoice_customerid ON Invoice(CustomerId);
CREATE INDEX IF NOT EXISTS idx_invoice_id ON Invoice(InvoiceId);
CREATE INDEX IF NOT EXISTS idx_invoiceline_invoiceid ON InvoiceLine(InvoiceId);
CREATE INDEX IF NOT EXISTS idx_invoiceline_trackid ON InvoiceLine(TrackId);
CREATE INDEX IF NOT EXISTS idx_track_id ON Track(TrackId);
""")
conn.commit()

print("Indexes created successfully.")

Indexes created successfully.


In [13]:
# %% [markdown]
# ## 7. Time the Query with Indexes

# %%
# First define the indexed query
indexed_query = """
SELECT 
    c.FirstName || ' ' || c.LastName AS CustomerName,
    i.InvoiceDate,
    t.Name AS TrackPurchased
FROM 
    Customer c
JOIN 
    Invoice i ON c.CustomerId = i.CustomerId
JOIN 
    InvoiceLine il ON i.InvoiceId = il.InvoiceId
JOIN 
    Track t ON il.TrackId = t.TrackId
ORDER BY 
    i.InvoiceDate DESC
LIMIT 10;
"""

# Then time the query
indexed_times = time_query(indexed_query)
avg_indexed = sum(indexed_times) / len(indexed_times)

print(f"Indexed query times (seconds): {[round(t, 4) for t in indexed_times]}")
print(f"Average time: {avg_indexed:.4f} seconds")
print(f"Performance improvement: {(avg_unindexed - avg_indexed)/avg_unindexed*100:.2f}% faster")

# %% [markdown]


Indexed query times (seconds): [0.0038, 0.0023, 0.0021, 0.0021, 0.0022, 0.0023, 0.0024, 0.0036, 0.0037, 0.0037]
Average time: 0.0028 seconds
Performance improvement: 99.05% faster


In [14]:
# ## 8. Examine Query Plans

# %%
# Make sure to include the indexed_query definition again or run the previous cell first
indexed_query = """
SELECT 
    c.FirstName || ' ' || c.LastName AS CustomerName,
    i.InvoiceDate,
    t.Name AS TrackPurchased
FROM 
    Customer c
JOIN 
    Invoice i ON c.CustomerId = i.CustomerId
JOIN 
    InvoiceLine il ON i.InvoiceId = il.InvoiceId
JOIN 
    Track t ON il.TrackId = t.TrackId
ORDER BY 
    i.InvoiceDate DESC
LIMIT 10;
"""

print("Query plan for UNINDEXED query:")
cursor.execute("EXPLAIN QUERY PLAN " + unindexed_query)
display(pd.DataFrame(cursor.fetchall(), columns=['id', 'parent', 'notused', 'detail']))

print("\nQuery plan for INDEXED query:")
cursor.execute("EXPLAIN QUERY PLAN " + indexed_query)
display(pd.DataFrame(cursor.fetchall(), columns=['id', 'parent', 'notused', 'detail']))

Query plan for UNINDEXED query:


,id,parent,notused,detail
0,7,0,0,SCAN TABLE Customer AS c
1,9,0,0,SCAN TABLE Invoice AS i
2,16,0,0,SCAN TABLE InvoiceLine AS il
3,23,0,0,SCAN TABLE Track AS t
4,47,0,0,USE TEMP B-TREE FOR ORDER BY



Query plan for INDEXED query:


,id,parent,notused,detail
0,7,0,0,SCAN TABLE InvoiceLine AS il
1,9,0,0,SEARCH TABLE Invoice AS i USING INTEGER PRIMAR...
2,12,0,0,SEARCH TABLE Customer AS c USING INTEGER PRIMA...
3,15,0,0,SEARCH TABLE Track AS t USING INTEGER PRIMARY ...
4,32,0,0,USE TEMP B-TREE FOR ORDER BY
